In [ ]:
from __future__ import print_function
import sisl
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In this example you will learn how to make use of the periodicity of the electrodes.

As seen in [example 04](../04/run.ipynb) the transmission calculation takes a considerable amount of time. In this example we will redo the *same* calculation, but speed it up (no approximations made).

A large computational effort is made on calculating the self-energies which basically is inverting, multiplying and adding matrices, roughly 10-20 times per $\mathbf k$-point, per energy point, per electrode.  
For some systems this is far more demanding than calculating the Green function for the system.  
In systems where there is periodicity along the transverse semi-infinite direction (not along the transport direction) one can utilize Bloch's theorem to reduce the computational cost of calculating the self-energy. In ***ANY*** calculation if you have periodicity, please ***USE*** it. 

In this example you should scour the tbtrans manual on how to enable Bloch's 
theorem, and once enabled it should be roughly 3 - 4 times as fast, something that is non-negligeble for large systems.

In [ ]:
graphene = sisl.geom.graphene(orthogonal=True)

Note the below lines are differing from the same lines in [example 04](../04/run.ipynb), i.e. we save the electrode electronic structure *before* extending it 25 times.

In [ ]:
H = sisl.Hamiltonian(graphene)
H.construct(([0.1, 1.43], [0., -2.7]))
H.write('ELEC.nc')
elec = graphene.repeat(25, axis=0)

In [ ]:
device = elec.tile(15, axis=1)
device = device.remove(device.close(device.center(which='cell'), R=10.))

dangling = [ia for ia in device.close(device.center(which='cell'), R=14.)
                if len(device.close(ia, R=1.43)) < 3]
device = device.remove(dangling)
edge = []
for ia in device.close(device.center(which='cell'), R=14.):
    if len(device.close(ia, R=1.43)) < 4:
        edge.append(ia + 1) # + 1 to get fortran indices

# Pretty-print the list of atoms
print(sisl.utils.list2str(edge))

Hdev = sisl.Hamiltonian(device)
Hdev.construct(([0.1, 1.43], [0, -2.7]))
Hdev.geom.write('device.xyz')
Hdev.write('DEVICE.nc')

# Exercises

Instead of analysing the same thing as in [04](../04/run.ipynb) you should perform the following actions to explore the available data-analysis capabilities of TBtrans.

*HINT* please copy as much as you like from example 04 to simplify the following tasks.

1. Read in the resulting file into a variable called `tbt`.
2. In the following we will concentrate on *only* looking at $\Gamma$-point related quantities. I.e. all quantities should only be plotted for this $\mathbf k$-point.  
   To extract information for one or more subset of points you should look into the function
       
       help(tbt.kindex)
   
   which may be used to find a resulting $\mathbf k$-point index in the result file.
   
3. Plot the transmission ($\Gamma$-point only). To extract a subset $\mathbf k$-point you should read the documentation for the functions (*hint: `kavg` is the keyword you are looking for*).
   - Full transmission
   - Bulk transmission
4. Plot the DOS with normalization according to the number of atoms ($\Gamma$ only)  
   You may decide which atoms you examine.
   - The Green function DOS
   - The spectral DOS
   - The bulk DOS

### Transmission

### Density of states